In [1]:
import os.path
import logging
import numpy as np
import pandas as pd
import sqlite3

import metar_tools
import taf_tools
#import misc_tools

data_yrs = list(range(2011, 2021+1))

In [2]:
sect_df = pd.read_csv('data/FH-base_sektorit.csv', sep=',', index_col='Sector_no')
sect_df

,fid,Tukikohta,Kuvaus,distance,Sector,dep_dest,alt_open,sect_wx,alt_all,ASEMA,luokka,ELEV_FT,ruutu_id,min_ifr_ft,GND_max,cld_ceiling
Sector_no,,,,,,,,,,,,,,,,
FH40.5,6,FH40,Seinäjoen tukikohta (arvio),100000,5,EFSI,EFTP;EFJY,ILXN,EFVA;EFKK;EFHA;EFPO;EFTP;EFJY,Kankaanpää Niinisalo lentokenttä,AWS-asema,404,22-61.5,2100,153.632996,1700
FH20.3,8,FH20,Turun lentoasema,100000,3,EFTU,EFTP;EFHK,ILIK,EFPO;EFTP;EFMA;EFHK;EFHA;EFSI,Salo Kiikala lentokenttä,AWS-asema,384,23.5-60,2000,164.514999,1600
FH20.5,9,FH20,Turun lentoasema,100000,5,EFTU,EFTP;EFHK,ILZU,EFPO;EFTP;EFMA;EFHK;EFHA;EFSI,Parainen Utö,AWS-asema,20,21-59.5,1400,21.108000,1400
FH20.8,10,FH20,Turun lentoasema,100000,8,EFTU,EFTP;EFHK,ILXF,EFPO;EFTP;EFMA;EFHK;EFHA;EFSI,Rauma Kylmäpihlaja,AWS-asema,13,21-61,1700,35.522999,1700
FH20.7,11,FH20,Turun lentoasema,100000,7,EFTU,EFTP;EFHK,ILZI,EFPO;EFTP;EFMA;EFHK;EFHA;EFSI,Kustavi Isokari,AWS-asema,16,21-60.5,1700,47.653999,1700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FH60_1.2,171,FH60,"Kelloniemi, Kuopion tukikohta",100000,2,EFKU,EFJY;EFTP,ILXR,EFJO;EFJY;EFSA;EFMI;EFKI;EFHA,Rautavaara Ylä-Luosta,AWS-asema,538,28.5-63,2300,292.751007,1800
FH60_1.4,172,FH60,"Kelloniemi, Kuopion tukikohta",100000,4,EFKU,EFJY;EFTP,ILXC,EFJO;EFJY;EFSA;EFMI;EFKI;EFHA,Varkaus Kosulanniemi,AWS-asema,272,27.5-62,2200,199.429001,1900
FH60_2.8,173,FH60_level2,"Kelloniemi, Kuopion tukikohta",150000,8,EFKU,EFJY;EFTP,ILQY,EFJO;EFJY;EFSA;EFMI;EFKI;EFHA,Vaala Pelso,AWS-asema,371,26-64.5,1900,194.817001,1500


In [3]:
geo_df = pd.read_csv('data/Saahavaintoasemat.csv', sep=',', index_col='ICAO')
geo_df.loc['EFHK','LON']

24.95675

In [4]:
sector = sect_df.loc['FH10.2',] # Loviisan suuntaan
term = sector['dep_dest']
sect_wx = sector['sect_wx']

In [5]:
def load_metar_chunk(icao_code, year):
    cols = ['ttime', 'content']
    df = pd.read_csv(f'data/metar/{icao_code}_{year}_METAR.dat', sep=';', usecols=cols)
    df['time'] = pd.to_datetime(df['ttime'])
    return df.set_index('time')

In [6]:
def find_first_file(icao_code, type):
    for i, year in enumerate(data_yrs):
        if os.path.isfile(f'data/{type.lower()}/{icao_code}_{year}_{type}.dat'):
            return i
        logging.info(f'No {type} file for {icao_code} in year {year}.')
    raise RuntimeError("No data found")

In [7]:
def load_metar(icao_code):
    i = find_first_file(icao_code, 'METAR')
    metar = load_metar_chunk(icao_code, data_yrs[i])
    for year in data_yrs[i+1:]:
        try:
            df = load_metar_chunk(icao_code, year)
        except:
            logging.info(f'No METAR file for {icao_code} in year {year}.')
        metar = pd.concat((metar, df))
    return metar

In [8]:
def load_taf_chunk(icao_code, year):
    cols = ['ttime', 'content']
    df = pd.read_csv(f'data/taf/{icao_code}_{year}_TAF.dat', sep=';', usecols=cols)
    df['time'] = pd.to_datetime(df['ttime'])
    df.set_index('time', inplace=True)
    return df

In [9]:
def load_taf(icao_code):
    i = find_first_file(icao_code, 'TAF')
    taf = load_taf_chunk(icao_code, data_yrs[i])
    for year in data_yrs[i+1:]:
        try:
            df = load_taf_chunk(icao_code, year)
        except:
            logging.info(f'No TAF file for {icao_code} in year {year}.')
        taf = pd.concat((taf, df))
    taf = taf.asfreq('10min')
    ffill_limit = 200 # don't fill super long gaps. TAF time rolls over in a month,
                      # max validity 30 h so anything between 30 h and 1 month goes.
    taf['ttime'] = taf.index.strftime('%Y-%m-%d %H:%M:%S')
    taf['content'] = taf['content'].ffill(limit=ffill_limit)
    return taf.dropna()

In [10]:
# def _load_sector(name, sector_metadata):
#     metar_term = load_metar(sector_metadata.loc[name,'dep_dest'])
#     taf_term = load_taf(sector_metadata.loc[name,'dep_dest'])
#     metar_sect = load_metar(sector_metadata.loc[name,'sect_wx'])
#     return metar_term, taf_term, metar_sect

In [11]:
def extract_taf(data: np.ndarray):
    time_str = data[0]
    taf_str = data[1]
    forecast = taf_tools.parse(taf_str)
    icao = forecast.station
    vis = taf_tools.get_worstcase_vis(forecast, time_str, 3)
    return (icao, time_str, vis, taf_str)

In [12]:
def extract_metar(data: np.ndarray):
    time_str = data[0]
    metar_str = data[1]
    obs = metar_tools.parse(metar_str)
    temp = metar_tools.get_temp(obs)
    vis = metar_tools.get_vis(obs)
    rvr = metar_tools.get_rvr(obs)
    ceil = metar_tools.get_ceil(obs)
    base = metar_tools.get_base(obs, ceil)
    icao = metar_tools.get_icao(obs)
    night = misc_tools.is_night(time_str, geo_df.loc[icao,'LAT'], geo_df.loc[icao,'LON'])
    return (icao, time_str, temp, vis, rvr, ceil, base, night, metar_str)

In [13]:
# def transform_wx(metar_term, taf_term, metar_sect):
#     logging.info("Transforming terminal METAR")
#     metar_term_transformed = [extract_metar(d) for d in zip(metar_term.ttime, metar_term.content)]
#     logging.info("Transforming terminal TAF")
#     taf_term_transformed = [extract_taf(d) for d in zip(taf_term.ttime, taf_term.content)]
#     logging.info("Transforming sector METAR")
#     metar_sect_transformed = [extract_metar(d) for d in zip(metar_sect.ttime, metar_sect.content)]

#     metar_cols = ['icao', 'time', 'temp', 'vis', 'rvr', 'ceil', 'base', 'night', 'metar_msg']
#     metar_term = pd.DataFrame(metar_term_transformed, columns=metar_cols)
#     metar_term['time'] = pd.to_datetime(metar_term['time'])
#     metar_term.set_index('time', inplace=True)

#     metar_sect = pd.DataFrame(metar_sect_transformed, columns=metar_cols)
#     metar_sect['time'] = pd.to_datetime(metar_sect['time'])
#     metar_sect.set_index('time', inplace=True)

#     taf_cols = ['icao', 'time', 'taf_vis', 'taf_msg']
#     taf_term = pd.DataFrame(taf_term_transformed, columns=taf_cols)
#     taf_term['time'] = pd.to_datetime(taf_term['time'])
#     taf_term.set_index('time', inplace=True)

#     return metar_term, taf_term, metar_sect

In [14]:
def transform_metar(metar):
    metar_cols = ['icao', 'time', 'temp', 'vis', 'rvr', 'ceil', 'base', 'night', 'metar_msg']
    metar_trans = [extract_metar(d) for d in zip(metar.ttime, metar.content)]
    df = pd.DataFrame(metar_trans, columns=metar_cols)
    #df['time'] = pd.to_datetime(df['time'])
    #df.set_index('time', inplace=True)
    return df

In [15]:
def transform_taf(taf):
    taf_cols = ['icao', 'time', 'taf_vis', 'taf_msg']
    taf_trans = [extract_taf(d) for d in zip(taf.ttime, taf.content)]
    df = pd.DataFrame(taf_trans, columns=taf_cols)
    #df['time'] = pd.to_datetime(df['time'])
    #df.set_index('time', inplace=True)
    return df

In [16]:
#def merge_wx(metar_term, taf_term, metar_sect):
#    df = metar_term.join(taf_term, how='inner')
#    df = df.join(metar_sect, how='left', lsuffix='_term', rsuffix='_sect')
#    df.drop(columns=['night_term', 'rvr_sect'], inplace=True)
#    df.rename(columns={'night_sect': 'night'}, inplace=True)
#    return df

In [17]:
# def load_sector(name, sector_metadata):
#     metar_term, taf_term, metar_sect = _load_sector(name, sector_metadata)
#     metar_term, taf_term, metar_sect = transform_wx(metar_term, taf_term, metar_sect)
#     #df = merge_wx(metar_term, taf_term, metar_sect)
#     return metar_term, taf_term, metar_sect

In [18]:
def load_station(icao):
    metar = load_metar(icao)
    metar = transform_metar(metar)
    taf = load_taf(icao)
    taf = transform_taf(taf)
    return metar, taf

In [19]:
metar, taf = load_station('EFHK')

C:\Users\makes\miniconda3\envs\hems\lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body '04319//44' while processing 'EFHK 081020Z 18007KT 9999 BKN004 M02/M02 Q1015 04319//44 5439//36 BECMG BKN005='
  warnings.warn(message, RuntimeWarning)
C:\Users\makes\miniconda3\envs\hems\lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body '04319//44' while processing 'EFHK 081050Z 18005KT 120V200 9999 BKN005 M01/M02 Q1015 04319//44 5439//36 NOSIG='
  warnings.warn(message, RuntimeWarning)
C:\Users\makes\miniconda3\envs\hems\lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body '04319//44' while processing 'EFHK 081120Z 17005KT 9999 SCT005 BKN006 M01/M02 Q1015 04319//44 5439//36 TEMPO BKN010='
  warnings.warn(message, RuntimeWarning)
C:\Users\makes\miniconda3\envs\hems\lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body '154490194' while processing 'EFHK 221020Z 16005KT 3000 -SN BKN007 BKN011 M05/M0

In [20]:
metar

,icao,time,temp,vis,rvr,ceil,base,night,metar_msg
0,EFHK,2011-01-01 00:20:00,-2.0,10000,9999,600,600,True,EFHK 010020Z 15010KT 9999 -SG BKN006 M02/M03 Q...
1,EFHK,2011-01-01 00:50:00,-2.0,10000,9999,600,600,True,EFHK 010050Z 14009KT 9999 BKN006 M02/M03 Q0993...
2,EFHK,2011-01-01 01:20:00,-2.0,10000,9999,700,700,True,EFHK 010120Z 15010KT 9999 BKN007 BKN032 M02/M0...
3,EFHK,2011-01-01 01:50:00,-2.0,10000,9999,700,700,True,EFHK 010150Z 14010KT 9999 BKN007 BKN025 M02/M0...
4,EFHK,2011-01-01 02:20:00,-3.0,10000,9999,500,500,True,EFHK 010220Z 13011KT 9999 BKN005 M03/M04 Q0992...
...,...,...,...,...,...,...,...,...,...
189122,EFHK,2021-12-03 08:20:00,-12.0,6000,9999,700,700,False,EFHK 030820Z 01008KT 6000 -SN BKN007 M12/M14 Q...
189123,EFHK,2021-12-03 08:50:00,-12.0,7000,9999,9999,400,False,EFHK 030850Z 35006KT 7000 -SN FEW004 SCT007 M1...
189124,EFHK,2021-12-03 09:20:00,-12.0,9000,9999,9999,400,False,EFHK 030920Z 36007KT 9000 -SN FEW004 M12/M13 Q...
189125,EFHK,2021-12-03 09:50:00,-12.0,10000,9999,9999,400,False,EFHK 030950Z 36008KT 9999 -SN FEW004 SCT022 M1...


In [21]:
taf

,icao,time,taf_vis,taf_msg
0,EFHK,2011-01-01 00:00:00,3000.0,EFHK 312340Z 0100/0124 11007KT 9999 SCT004 BKN...
1,EFHK,2011-01-01 00:10:00,3000.0,EFHK 312340Z 0100/0124 11007KT 9999 SCT004 BKN...
2,EFHK,2011-01-01 00:20:00,3000.0,EFHK 312340Z 0100/0124 11007KT 9999 SCT004 BKN...
3,EFHK,2011-01-01 00:30:00,3000.0,EFHK 312340Z 0100/0124 11007KT 9999 SCT004 BKN...
4,EFHK,2011-01-01 00:40:00,3000.0,EFHK 312340Z 0100/0124 11007KT 9999 SCT004 BKN...
...,...,...,...,...
570143,EFHK,2021-12-08 13:20:00,2000.0,EFHK 081137Z 0812/0912 32003KT 9999 FEW001 PRO...
570144,EFHK,2021-12-08 13:30:00,2000.0,EFHK 081137Z 0812/0912 32003KT 9999 FEW001 PRO...
570145,EFHK,2021-12-08 13:40:00,2000.0,EFHK 081137Z 0812/0912 32003KT 9999 FEW001 PRO...
570146,EFHK,2021-12-08 13:50:00,2000.0,EFHK 081137Z 0812/0912 32003KT 9999 FEW001 PRO...


In [26]:
con = sqlite3.connect("hems.sqlite")

In [ ]:
metar.to_sql("t_metar", con, if_exists='append')

In [23]:
taf.to_sql("t_taf", con, if_exists='append')

In [24]:
metar

,icao,time,temp,vis,rvr,ceil,base,night,metar_msg
0,EFHK,2011-01-01 00:20:00,-2.0,10000,9999,600,600,True,EFHK 010020Z 15010KT 9999 -SG BKN006 M02/M03 Q...
1,EFHK,2011-01-01 00:50:00,-2.0,10000,9999,600,600,True,EFHK 010050Z 14009KT 9999 BKN006 M02/M03 Q0993...
2,EFHK,2011-01-01 01:20:00,-2.0,10000,9999,700,700,True,EFHK 010120Z 15010KT 9999 BKN007 BKN032 M02/M0...
3,EFHK,2011-01-01 01:50:00,-2.0,10000,9999,700,700,True,EFHK 010150Z 14010KT 9999 BKN007 BKN025 M02/M0...
4,EFHK,2011-01-01 02:20:00,-3.0,10000,9999,500,500,True,EFHK 010220Z 13011KT 9999 BKN005 M03/M04 Q0992...
...,...,...,...,...,...,...,...,...,...
189122,EFHK,2021-12-03 08:20:00,-12.0,6000,9999,700,700,False,EFHK 030820Z 01008KT 6000 -SN BKN007 M12/M14 Q...
189123,EFHK,2021-12-03 08:50:00,-12.0,7000,9999,9999,400,False,EFHK 030850Z 35006KT 7000 -SN FEW004 SCT007 M1...
189124,EFHK,2021-12-03 09:20:00,-12.0,9000,9999,9999,400,False,EFHK 030920Z 36007KT 9000 -SN FEW004 M12/M13 Q...
189125,EFHK,2021-12-03 09:50:00,-12.0,10000,9999,9999,400,False,EFHK 030950Z 36008KT 9999 -SN FEW004 SCT022 M1...


In [27]:
con.close()

In [30]:
for icao in geo_df.index:
    print(icao)

EFET
EFHA
EFHK
EFIV
EFJO
EFJY
EFKE
EFKI
EFKK
EFKS
EFKT
EFKU
EFLP
EFMA
EFMI
EFOU
EFPO
EFRO
EFSA
EFSI
EFTP
EFTU
EFUT
EFVA
ILHF
ILIK
ILJY
ILKA
ILKM
ILMI
ILPR
ILPU
ILQA
ILQB
ILQC
ILQD
ILQE
ILQF
ILQH
ILQI
ILQJ
ILQK
ILQL
ILQM
ILQN
ILQO
ILQP
ILQQ
ILQR
ILQS
ILQT
ILQU
ILQV
ILQX
ILQY
ILRO
ILRU
ILWA
ILWD
ILWE
ILWF
ILWG
ILXA
ILXB
ILXC
ILXD
ILXE
ILXF
ILXG
ILXI
ILXJ
ILXK
ILXL
ILXM
ILXN
ILXO
ILXP
ILXQ
ILXR
ILXS
ILXT
ILXU
ILXV
ILXW
ILXX
ILXY
ILXZ
ILYL
ILZA
ILZB
ILZC
ILZD
ILZE
ILZF
ILZG
ILZH
ILZI
ILZJ
ILZK
ILZL
ILZM
ILZO
ILZP
ILZQ
ILZR
ILZS
ILZT
ILZU
ILZV
ILZW
ILZX
ILZY
ILZZ
